In [75]:
import numpy as np
import matplotlib.pyplot as plt
from boundaries import WallBoundary

# Initializing a rank three array filled with nodes
latticeSize = 9
xResolution = 100
yResolution = 100
# Weights
cxs = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1])
cys = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1])
weight = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36])

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 13)

In [74]:
# One iteration only


NameError: name 'it' is not defined